In [2]:
%reset -f
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import xarray as xr
import h5py

import matplotlib.pyplot as plt
import sys
sys.path.append("./lib")
from preprocess import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
output_folder = "../data/preprocessed/"
file_format= "%s"
parameters = {"nan_dim_along":"time", "nan_dim_irrelevant":"absolute_height", "output_folder":output_folder,"file_format":file_format}
list_of_data_set_path=['../data/perdigao_era5_2020.nc', '../data/perdigao_low_res_1H_2020.nc', '../data/perdigao_high_res_1H_2020.nc' ]
data_preprocess(list_of_data_set_path, parameters)

Creating DataSets by loading files:['../data/perdigao_era5_2020.nc', '../data/perdigao_low_res_1H_2020.nc', '../data/perdigao_high_res_1H_2020.nc']
Removing NAN indices along dimension:time
WARNING! This will change the oringal DataSets!
Searching NAN in DataSets: dict_keys(['perdigao_era5_2020', 'perdigao_low_res_1H_2020', 'perdigao_high_res_1H_2020'])...
Checking nan pattern of variable: u100  for DataSet: perdigao_era5_2020
Total number of NAN: (0,), along (0,) time indicies
Checking nan pattern of variable: v100  for DataSet: perdigao_era5_2020
Total number of NAN: (0,), along (0,) time indicies
Checking nan pattern of variable: t2m  for DataSet: perdigao_era5_2020
Total number of NAN: (0,), along (0,) time indicies
Checking nan pattern of variable: i10fg  for DataSet: perdigao_era5_2020
Total number of NAN: (0,), along (0,) time indicies
NAN pattern along dimension: time, is CONSISTENT for all other coords, with absolute_height excluded
Checking nan pattern of variable: std  for D

# Compute the statistics of low and high resolution data for GAN

In [ ]:
mean_lr_u = ds_low_res["u"].mean().to_numpy()
mean_lr_v = ds_low_res["v"].mean().to_numpy()
mean_hr_u = ds_high_res["u"].mean().to_numpy()
mean_hr_v = ds_high_res["v"].mean().to_numpy()

In [ ]:
print(mean_lr_u)
print(mean_lr_v)
print(mean_hr_u)
print(mean_hr_v)

In [ ]:
stddev_lr_u = ds_low_res["u"].std().to_numpy()
stddev_lr_v = ds_low_res["v"].std().to_numpy()
stddev_hr_u = ds_high_res["u"].std().to_numpy()
stddev_hr_v = ds_high_res["v"].std().to_numpy()

In [ ]:
print(stddev_lr_u)
print(stddev_lr_v)
print(stddev_hr_u)
print(stddev_hr_v)

# Standardrize the dataset for GAN

In [ ]:
da_high_res_std_u = ( ds_high_res["u"] - mean_hr_u ) / stddev_hr_u
da_high_res_std_v = ( ds_high_res["v"] - mean_hr_v ) / stddev_hr_v
print(da_high_res_std_u.mean().to_numpy())
print(da_high_res_std_v.mean().to_numpy())
print(da_high_res_std_u.std().to_numpy())
print(da_high_res_std_v.std().to_numpy())

In [ ]:
#so weird... If I stack them with axis=-1, then result of std and mean will be incorrect!
np_hr_std_pre = np.stack((da_high_res_std_u.to_numpy(), da_high_res_std_v.to_numpy()), axis=0)
print(np_hr_std_pre.shape)

np_hr_std = np_hr_std_pre.transpose(1,2,3,0)
print(np_hr_std.shape)

print(np_hr_std.std(axis=(0,1,2)))
print(np_hr_std.mean(axis=(0,1,2)))

In [ ]:
da_low_res_std_u = ( ds_low_res["u"] - mean_lr_u ) / stddev_lr_u
da_low_res_std_v = ( ds_low_res["v"] - mean_lr_v ) / stddev_lr_v
print(da_low_res_std_u.mean().to_numpy())
print(da_low_res_std_v.mean().to_numpy())
print(da_low_res_std_u.std().to_numpy())
print(da_low_res_std_v.std().to_numpy())

In [ ]:
#so weird... If I stack them with axis=-1, then result of std and mean will be incorrect!
np_lr_std_pre = np.stack((da_low_res_std_u.to_numpy(), da_low_res_std_v.to_numpy()), axis=0)
print(np_lr_std_pre.shape)

np_lr_std = np_lr_std_pre.transpose(1,2,3,0)
print(np_lr_std.shape)

print(np_lr_std.std(axis=(0,1,2)))
print(np_lr_std.mean(axis=(0,1,2)))

In [ ]:
lr_mean = np.array([mean_lr_u, mean_lr_v])
hr_mean = np.array([mean_hr_u, mean_hr_v])
lr_stddev = np.array([stddev_lr_u, stddev_lr_v])
hr_stddev = np.array([stddev_hr_u, stddev_hr_v])

In [ ]:
with h5py.File('processed_data/np_gan_standard.h5', 'w') as hf:
    hf.create_dataset("np_lr",  data=np_lr_std)
    hf.create_dataset("np_hr",  data=np_hr_std)
    hf.create_dataset("np_lr_mean",  data=lr_mean)
    hf.create_dataset("np_hr_mean",  data=hr_mean)
    hf.create_dataset("np_lr_stddev",  data=lr_stddev)
    hf.create_dataset("np_hr_stddev",  data=hr_stddev)

In [ ]:
with h5py.File('processed_data/np_gan_standard.h5', 'r') as hf:
    data_lr = hf['np_lr'][:]
    data_lr_mean = hf['np_lr_mean'][:]
    data_lr_stddev = hf['np_lr_stddev'][:]
    data_hr = hf['np_hr'][:]
    data_hr_mean = hf['np_hr_mean'][:]
    data_hr_stddev = hf['np_hr_stddev'][:]

In [ ]:
print(np.array_equal(np_lr_std, data_lr))

In [ ]:
print(np.array_equal(np_hr_std, data_hr))

In [ ]:
np_lr_reopen = data_lr * data_lr_stddev + data_lr_mean
print(np.max(np_lr_reopen[:,:,:,0] - ds_low_res["u"].to_numpy()))
print(np.max(np_lr_reopen[:,:,:,1] - ds_low_res["v"].to_numpy()))

In [ ]:
np_hr_reopen = data_hr * data_hr_stddev + data_hr_mean
print(np.max(np_hr_reopen[:,:,:,0] - ds_high_res["u"].to_numpy()))
print(np.max(np_hr_reopen[:,:,:,1] - ds_high_res["v"].to_numpy()))